In [10]:
import pandas as pd
import json
import re

In [11]:
# Load JSON data (replace 'data.json' with the actual file path)
with open("trials.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

In [5]:
df = pd.DataFrame([{
    "RCT_ID": item.get("RCT ID", None),
    "Title": item.get("Title", None),
    "Status": item.get("Status", None),
    "Start_Date": item.get("Start date", None),
    "End_Date": item.get("End date", None),
    "Country": ", ".join([c["Country"] if isinstance(c, dict) else str(c) for c in item.get("Countries", [])]) if "Countries" in item else None,
    "Region": ", ".join([c["Region"] if isinstance(c, dict) and isinstance(c["Region"], str) else str(c["Region"]) if isinstance(c["Region"], list) else "" for c in item.get("Countries", [])]) if "Countries" in item else None,
    "PI_Name": item.get("Primary PI", {}).get("Name", None),
    "PI_Affiliation": item.get("Primary PI", {}).get("Affiliation", None),
    "Keywords": ", ".join(item.get("Keywords", [])) if item.get("Keywords") else None,
    "Additional_Keywords": ", ".join(item.get("Additional Keywords", [])) if item.get("Additional Keywords") else None,
    "JEL_Codes": ", ".join(item.get("JEL code(s)", [])) if item.get("JEL code(s)") else None,
    "Secondary_IDs": ", ".join(item.get("Secondary IDs", [])) if item.get("Secondary IDs") else None,
    "Abstract": item.get("Abstract", None),
    "External_Links": ", ".join(item.get("External Link(s)", {}).get("Link", [])) if isinstance(item.get("External Link(s)"), dict) else None,
    "Citation": item.get("Citation", None),
    "Former_Citation": item.get("Former Citation", None),
    "Intervention": item.get("Intervention(s)", None),
    "Intervention_Hidden": item.get("Intervention (Hidden)", None),
    "Intervention_Start_Date": item.get("Intervention Start Date", None),
    "Intervention_End_Date": item.get("Intervention End Date", None),
    "Primary_Outcomes": ", ".join(item.get("Primary Outcomes (end points)", [])) if item.get("Primary Outcomes (end points)") else None,
    "Primary_Outcomes_Explanation": item.get("Primary Outcomes (explanation)", None),
    "Secondary_Outcomes": ", ".join(item.get("Secondary Outcomes (end points)", [])) if item.get("Secondary Outcomes (end points)") else None,
    "Secondary_Outcomes_Explanation": item.get("Secondary Outcomes (explanation)", None),
    "Experimental_Design": item.get("Experimental Design", None),
    "Experimental_Design_Details": item.get("Experimental Design Details", None),
    "Randomization_Method": item.get("Randomization Method", None),
    "Randomization_Unit": item.get("Randomization Unit", None),
    "Treatment_Clustered": item.get("Was the treatment clustered?", None),
    "Planned_Clusters": item.get("Sample size: planned number of clusters", None),
    "Planned_Observations": item.get("Sample size: planned number of observations", None),
    "Sample_By_Treatment_Arms": item.get("Sample size (or number of clusters) by treatment arms", None),
    "Minimum_Detectable_Effect": item.get("Minimum detectable effect size for main outcomes (accounting for sampledesign and clustering)", None),
    "Intervention_Completed": item.get("Is the intervention completed?", None),
    "Data_Collection_Complete": item.get("Data Collection Complete", None),
    "Public_Data_Available": item.get("Is public data available?", None),
    "Program_Files": item.get("Program Files", None)
} for item in json_data.values()])

# Extract multiple investigators separately
df_pis = pd.concat([
    pd.DataFrame({
        "RCT_ID": item.get("RCT ID", None),
        "PI_Name": [pi.get("Name", None) for pi in item.get("Other Primary Investigators", [])],
        "PI_Affiliation": [pi.get("Affiliation", None) for pi in item.get("Other Primary Investigators", [])]
    }) for item in json_data.values() if "Other Primary Investigators" in item
], ignore_index=True)

# Extract IRB approvals separately
df_irbs = pd.concat([
    pd.DataFrame({
        "RCT_ID": item.get("RCT ID", None),
        "IRB_Name": [irb.get("Name", None) for irb in item.get("IRBs", [])],
        "IRB_Approval_Date": [irb.get("Approval Date", None) for irb in item.get("IRBs", [])],
        "IRB_Approval_Number": [irb.get("Approval Number", None) for irb in item.get("IRBs", [])]
    }) for item in json_data.values() if "IRBs" in item
], ignore_index=True)

In [13]:
df.columns

Index(['RCT_ID', 'Title', 'Status', 'Start_Date', 'End_Date', 'Country',
       'Region', 'PI_Name', 'PI_Affiliation', 'Keywords',
       'Additional_Keywords', 'JEL_Codes', 'Secondary_IDs', 'Abstract',
       'External_Links', 'Citation', 'Former_Citation', 'Intervention',
       'Intervention_Hidden', 'Intervention_Start_Date',
       'Intervention_End_Date', 'Primary_Outcomes',
       'Primary_Outcomes_Explanation', 'Secondary_Outcomes',
       'Secondary_Outcomes_Explanation', 'Experimental_Design',
       'Experimental_Design_Details', 'Randomization_Method',
       'Randomization_Unit', 'Treatment_Clustered', 'Planned_Clusters',
       'Planned_Observations', 'Sample_By_Treatment_Arms',
       'Minimum_Detectable_Effect', 'Intervention_Completed',
       'Data_Collection_Complete', 'Public_Data_Available', 'Program_Files'],
      dtype='object')

In [12]:
def extract_single_number(text):
    numbers = re.findall(r'\b\d{1,3}(?:[, ]\d{3})*\b', text)  # Extract numbers formatted with commas or spaces
    numbers = [int(num.replace(',', '').replace(' ', '')) for num in numbers]  # Convert to integers

    return numbers[0] if len(numbers) == 1 else None  # Return only if there's exactly one number

In [16]:
df['sample_size'] = [extract_single_number(text) for text in df['Planned_Observations']]

In [32]:
df['Experimental_Design'].head()

0    Treatment was randomly assigned at the individ...
1    The study is a clustered randomized controlled...
2    This is a three group design:\r-control (98 sc...
3    Panchayats in randomly selected blocks will ei...
4    We randomly selected 62 villages participating...
Name: Experimental_Design, dtype: object

In [18]:
df['sample_size'].describe().round(2)

count        4676.00
mean        34517.91
std        961062.49
min             1.00
25%           300.00
50%           686.50
75%          3000.00
max      50000000.00
Name: sample_size, dtype: float64

In [30]:
df['Country'].value_counts()

Country
United States of America                                       2014
Germany                                                         711
India                                                           531
China                                                           350
United Kingdom of Great Britain and Northern Ireland            281
                                                               ... 
Azerbaijan                                                        1
India, Norway                                                     1
Germany, Denmark, Estonia, Spain, Finland, Italy, Lithuania       1
Serbia, Turkey, Ukraine                                           1
Netherlands, United States of America                             1
Name: count, Length: 498, dtype: int64

In [19]:
df['Planned_Observations'][df['sample_size'] < 10]

19                          Approximately 5 million people
63                                9 households per village
517      4096 individuals. See Section 3 of our (public...
619                                                      2
650      The total sample size is approximately 3000 wo...
                               ...                        
9912                                                     1
9913            3000 (3 scenario responses by respondent).
9934                             7 households per village.
9973              10000 observations; 5 rounds per subject
10008    13500: 4500 individuals times 3 choice sets each 
Name: Planned_Observations, Length: 94, dtype: object

In [20]:
df['Sample_By_Treatment_Arms']

0                                                      N/A
1                                24 clusters per study arm
2        98 school campuses control, 93 school campuses...
3                   69 treatment blocks,127 control blocks
4        2821 individuals free DFS,  3224 individuals n...
                               ...                        
10051    1,900 control, 744 ESG messages, 736 E message...
10052                                                    \
10053                                                   76
10054    The number of participants for each round is a...
10055          750 students per group (treatment, control)
Name: Sample_By_Treatment_Arms, Length: 10056, dtype: object

In [22]:
list(df['Minimum_Detectable_Effect'])

['MDE=5% change in perceived leakage, sd=28.8667, power=0.80, alpha=0.05',
 '',
 '',
 '',
 '',
 '',
 '0.7 g/dL among elderly subgroup',
 '',
 '',
 '0.2 SD, 80%',
 '0.2 SD, 89%',
 'Please see the separate Power Calculations document under Supporting Documents &amp; Materials.',
 '',
 '',
 '',
 'With alpha = 0.05, within cluster correlation of 0.10, 1 Standard Deviation and 12 control cluster and 24 treatment cluster, our Minimum Detectable Effect Size for Main Outcomes would be 0.396 ',
 'For Part II: MDE is 7.7 ppt increase in voting across party lines with alpha 0.05 and power 0.80; Part III: same MDE as in II for T versus C as well as continuous outcome MDE of 0.5 point shift on 10 point likeability scale across T arms',
 '',
 '',
 '1.5 percentage points in voter turnout',
 '',
 '',
 '',
 '',
 '',
 '',
 '80% power for all:  Violent Arrests (0.058 arrests, 0.364 SD); Property Arrests (0.043 arrests, 0.266 SD); Drug Arrests (0.091 arrests, 0.570 SD); Other Arrests (0.121 arrests, 0.759

In [23]:
df.shape

(10056, 39)